In [4]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# =============================================================================
# 1. DATEN LADEN UND VORBEREITEN
# =============================================================================

def load_and_prepare_data():
    """Lädt die Daten und bereitet sie für das Modelling vor."""
    print("Lade Daten...")
    
    # Daten laden
    df = pd.read_csv('/workspaces/bakery_sales_prediction/2_BaselineModel/data/data_clean.csv')
    sample_sub = pd.read_csv('/workspaces/bakery_sales_prediction/2_BaselineModel/datasets/sample_submission.csv')
    
    # Datum konvertieren
    df['Datum'] = pd.to_datetime(df['Datum'])
    
    # Zusätzliche Zeit-Features erstellen
    df['Tag'] = df['Datum'].dt.day
    df['Monat'] = df['Datum'].dt.month
    df['Jahr'] = df['Datum'].dt.year
    df['Wochentag'] = df['Datum'].dt.dayofweek  # 0=Montag, 6=Sonntag
    
    print(f"Daten geladen: {len(df)} Datensätze von {df['Datum'].min()} bis {df['Datum'].max()}")
    print(f"Sample Submission: {len(sample_sub)} Vorhersagen benötigt")
    
    return df, sample_sub

# =============================================================================
# 2. DATEN AUFTEILEN
# =============================================================================

def split_data(df):
    """Teilt die Daten in Train/Validation/Test auf."""
    print("\nTeile Daten auf...")
    
    # Zeitbasierte Aufteilung
    train = df[df['Datum'] < '2017-08-01'].copy()
    validation = df[(df['Datum'] >= '2017-08-01') & (df['Datum'] < '2018-08-01')].copy()
    
    print(f"Training: {len(train)} Datensätze ({train['Datum'].min()} - {train['Datum'].max()})")
    print(f"Validation: {len(validation)} Datensätze ({validation['Datum'].min()} - {validation['Datum'].max()})")
    
    return train, validation

# =============================================================================
# 3. FEATURES DEFINIEREN
# =============================================================================

def get_features():
    """Definiert die Features für die Regression."""
    # Wetter-Features
    weather_features = [
        'Bewoelkung', 'Temperatur', 'Windgeschwindigkeit', 'Wettercode',
        'Wettercode_fehlt'
    ]
    
    # Feiertags-Features
    holiday_features = [
        'ist_feiertag', 'feiertag_vortag', 'feiertag_folgetag',
        'KielerWoche'
    ]
    
    # Zeit-Features
    time_features = [
        'Tag', 'Monat', 'Jahr', 'Wochentag'
    ]
    
    # Alle Features kombinieren
    features = weather_features + holiday_features + time_features
    
    print(f"Verwende {len(features)} Features: {features}")
    return features

# =============================================================================
# 4. WARENGRUPPEN IDENTIFIZIEREN
# =============================================================================

def get_product_groups(df):
    """Identifiziert aktive Warengruppen in den Daten."""
    product_columns = [col for col in df.columns if col.startswith('Warengruppe_')]
    
    active_groups = []
    for col in product_columns:
        if df[col].sum() > 0:  # Nur Warengruppen mit Daten
            group_name = col.replace('Warengruppe_', '')
            active_groups.append((group_name, col))
    
    print(f"\nAktive Warengruppen: {[group[0] for group in active_groups]}")
    return active_groups

# =============================================================================
# 5. MODELL TRAINING
# =============================================================================

def train_models(train_data, validation_data, features, product_groups):
    """Trainiert separate Modelle für jede Warengruppe."""
    print("\n" + "="*50)
    print("MODELL TRAINING")
    print("="*50)
    
    models = {}
    results = {}
    
    # Listen für Gesamtvorhersagen und tatsächliche Werte
    all_y_true = []
    all_y_pred = []
    
    for group_name, group_col in product_groups:
        print(f"\nTrainiere Modell für {group_name}...")
        
        # Daten für diese Warengruppe filtern
        train_group = train_data[train_data[group_col] == 1].copy()
        val_group = validation_data[validation_data[group_col] == 1].copy()
        
        if len(train_group) == 0:
            print(f"  Keine Trainingsdaten für {group_name}")
            continue
            
        # Features und Target vorbereiten
        X_train = train_group[features]
        y_train = train_group['Umsatz']
        
        # Modell trainieren
        model = LinearRegression()
        model.fit(X_train, y_train)
        
        # Validierung
        if len(val_group) > 0:
            X_val = val_group[features]
            y_val = val_group['Umsatz']
            
            y_pred = model.predict(X_val)
            mae = mean_absolute_error(y_val, y_pred)
            rmse = np.sqrt(mean_squared_error(y_val, y_pred))
            r2 = r2_score(y_val, y_pred)
            
            # Für Gesamtmetrik sammeln
            all_y_true.extend(y_val)
            all_y_pred.extend(y_pred)
            
            results[group_name] = {
                'mae': mae,
                'rmse': rmse,
                'r2': r2,
                'train_samples': len(train_group),
                'val_samples': len(val_group)
            }
            
            print(f"  Training: {len(train_group)} Datensätze")
            print(f"  Validation: {len(val_group)} Datensätze")
            print(f"  MAE: {mae:.2f}")
            print(f"  RMSE: {rmse:.2f}")
            print(f"  R²: {r2:.4f}")
        
        models[group_name] = model
    
    # Gesamtmetriken berechnen
    if all_y_true and all_y_pred:
        total_r2 = r2_score(all_y_true, all_y_pred)
        results['gesamt'] = {
            'r2': total_r2
        }
        print(f"\nGesamtperformance über alle Warengruppen:")
        print(f"  R²: {total_r2:.4f}")
    
    return models, results

# =============================================================================
# 6. VORHERSAGEN ERSTELLEN
# =============================================================================

def create_predictions(models, sample_sub, features):
    """Erstellt Vorhersagen für das Test-Set."""
    print("\n" + "="*50)
    print("VORHERSAGEN ERSTELLEN")
    print("="*50)
    
    # Warengruppen-Mapping
    group_mapping = {
        '1': 'Brot',
        '2': 'Brötchen', 
        '3': 'Croissant',
        '4': 'Konditorei',
        '5': 'Kuchen',
        '6': 'Saisonbrot'
    }
    
    predictions = []
    
    for idx, row in sample_sub.iterrows():
        id_str = str(row['id'])
        
        # ID parsen: YYMMDD + Warengruppe
        if len(id_str) == 7:
            date_part = id_str[:6]  # YYMMDD
            group_code = id_str[6]  # Warengruppe
            
            # Datum rekonstruieren
            year = 2000 + int(date_part[:2])
            month = int(date_part[2:4])
            day = int(date_part[4:6])
            
            try:
                date = datetime(year, month, day)
                
                # Features für Vorhersage erstellen
                feature_dict = {
                    'Tag': day,
                    'Monat': month,
                    'Jahr': year,
                    'Wochentag': date.weekday(),
                    # Dummy-Werte für Wetter und Feiertage (könnten verbessert werden)
                    'Bewoelkung': 5.0,
                    'Temperatur': 15.0,
                    'Windgeschwindigkeit': 10.0,
                    'Wettercode': 20.0,
                    'Wettercode_fehlt': 0,
                    'ist_feiertag': 0,
                    'feiertag_vortag': 0,
                    'feiertag_folgetag': 0,
                    'KielerWoche': 0
                }
                
                # Vorhersage mit entsprechendem Modell
                group_name = group_mapping.get(group_code)
                if group_name and group_name in models:
                    X_pred = pd.DataFrame([feature_dict])[features]
                    prediction = models[group_name].predict(X_pred)[0]
                    prediction = max(0, prediction)  # Keine negativen Umsätze
                else:
                    prediction = 0  # Fallback
                
                predictions.append(prediction)
                
            except ValueError:
                predictions.append(0)  # Fallback bei ungültigem Datum
        else:
            predictions.append(0)  # Fallback bei ungültiger ID
    
    # Ergebnis-DataFrame erstellen
    result_df = sample_sub.copy()
    result_df['Umsatz'] = predictions
    
    print(f"Vorhersagen erstellt für {len(predictions)} IDs")
    print(f"Durchschnittlicher vorhergesagter Umsatz: {np.mean(predictions):.2f}")
    
    return result_df

# =============================================================================
# 7. HAUPTFUNKTION
# =============================================================================

def main():
    """Hauptfunktion für die komplette Pipeline."""
    print("BÄCKEREI UMSATZ VORHERSAGE")
    print("="*50)
    
    # 1. Daten laden
    df, sample_sub = load_and_prepare_data()
    
    # 2. Daten aufteilen
    train_data, validation_data = split_data(df)
    
    # 3. Features definieren
    features = get_features()
    
    # 4. Warengruppen identifizieren
    product_groups = get_product_groups(df)
    
    # 5. Modelle trainieren
    models, results = train_models(train_data, validation_data, features, product_groups)
    
    # 6. Vorhersagen erstellen
    predictions_df = create_predictions(models, sample_sub, features)
    
    # 7. Ergebnisse speichern
    output_path = '/workspaces/bakery_sales_prediction/2_BaselineModel/predictions_linear_regression.csv'
    predictions_df.to_csv(output_path, index=False)
    
    print(f"\n" + "="*50)
    print("FERTIG!")
    print("="*50)
    print(f"Vorhersagen gespeichert in: {output_path}")
    
    # Zusammenfassung der Modell-Performance
    if results:
        print("\nModell-Performance (Validation):")
        for group, metrics in results.items():
            if group != 'gesamt':
                print(f"  {group}: MAE={metrics['mae']:.2f}, RMSE={metrics['rmse']:.2f}, R²={metrics['r2']:.4f}")
        
        # Gesamtperformance ausgeben
        if 'gesamt' in results:
            print(f"\nGesamtperformance: R²={results['gesamt']['r2']:.4f}")

# =============================================================================
# AUSFÜHRUNG
# =============================================================================

if __name__ == "__main__":
    main()

BÄCKEREI UMSATZ VORHERSAGE
Lade Daten...
Daten geladen: 9334 Datensätze von 2013-07-01 00:00:00 bis 2018-07-31 00:00:00
Sample Submission: 1830 Vorhersagen benötigt

Teile Daten auf...
Training: 7493 Datensätze (2013-07-01 00:00:00 - 2017-07-31 00:00:00)
Validation: 1841 Datensätze (2017-08-01 00:00:00 - 2018-07-31 00:00:00)
Verwende 13 Features: ['Bewoelkung', 'Temperatur', 'Windgeschwindigkeit', 'Wettercode', 'Wettercode_fehlt', 'ist_feiertag', 'feiertag_vortag', 'feiertag_folgetag', 'KielerWoche', 'Tag', 'Monat', 'Jahr', 'Wochentag']

Aktive Warengruppen: ['Brot', 'Brötchen', 'Croissant', 'Konditorei', 'Kuchen', 'Saisonbrot']

MODELL TRAINING

Trainiere Modell für Brot...
  Training: 1462 Datensätze
  Validation: 357 Datensätze
  MAE: 29.85
  RMSE: 43.72
  R²: -0.0899

Trainiere Modell für Brötchen...
  Training: 1462 Datensätze
  Validation: 357 Datensätze
  MAE: 51.00
  RMSE: 64.90
  R²: 0.5828

Trainiere Modell für Croissant...
  Training: 1462 Datensätze
  Validation: 357 Datens